In [1]:
import tensorflow as tf
import pandas as pd

diabetes = pd.read_csv('diabetes.csv')
diabetes.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
columns_to_norm = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction']

In [3]:
diabetes[columns_to_norm] = diabetes[columns_to_norm].apply(lambda x: (x - min(x)) / (x.max()-x.min() ))
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1


In [4]:
# Build continuous feature column

num_preg = tf.feature_column.numeric_column('Pregnancies')
plasma_gluc = tf.feature_column.numeric_column('Glucose')
dias_press = tf.feature_column.numeric_column('BloodPressure')
tricep = tf.feature_column.numeric_column('SkinThickness')
insulin = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
diabetes_pedigree = tf.feature_column.numeric_column('DiabetesPedigreeFunction')
age = tf.feature_column.numeric_column('Age')


In [5]:
# # Build categorical feature column
# assigned_group = tf.feature_column.categorical_column_with_vocabulary_list('Group',['A','B','C','D'])
# assigned_group = tf.feature_column.categorical_column_with_hash_bucket('Group',hash_bucket_size=10)

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
fig,ax=plt.subplots(figsize=(18,8))
sns.distplot(diabetes['Age'].dropna(),color='darkred',bins=30,ax=ax)

D:\Programme\Anaconda\envs\ialab\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [7]:
age_bucket = tf.feature_column.bucketized_column(age, boundaries=[20,30,40,50,60,70,80])

In [8]:
feat_cols = [num_preg,plasma_gluc,dias_press,tricep,insulin,bmi,diabetes_pedigree,age_bucket]

In [9]:
x_data = diabetes.drop('Outcome', axis=1)
labels = diabetes['Outcome']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.3)

In [10]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,batch_size=10,num_epochs=1000,shuffle=True)
model = tf.estimator.LinearClassifier(feature_columns=feat_cols, n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\matth\\AppData\\Local\\Temp\\tmpwnbsqwtf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018DD34AEB70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
model.train(input_fn=input_func, steps=1000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\matth\AppData\Local\Temp\tmpwnbsqwtf\model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 0
INFO:tensorflow:global_step/sec: 116.107
INFO:tensorflow:loss = 5.503206, step = 100 (0.852 sec)
INFO:tensorflow:global_step/sec: 151.688
INFO:tensorflow:loss = 7.194358, step = 200 (0.660 sec)
INFO:tensorflow:global_step/sec: 163.839
INFO:tensorflow:loss = 4.565102, step = 300 (0.610 sec)
INFO:tensorflow:global_step/sec: 140.825
INFO:tensorflow:loss = 4.79189, step

In [13]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=1000,num_epochs=1,shuffle=False)
result = model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-23-18:03:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\matth\AppData\Local\Temp\tmpwnbsqwtf\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-23-18:03:15
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7316017, accuracy_baseline = 0.6103896, auc = 0.7663121, auc_precision_recall = 0.7127547, average_loss = 0.56688124, global_step = 1000, label/mean = 0.38961038, loss = 130.94957, precision = 0.71875, prediction/mean = 0.35251445, recall = 0.51111114
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\matth\AppData\Local\Temp\tmpwnbsqwtf\model.ckpt-1000


In [14]:
result

{'accuracy': 0.7316017,
 'accuracy_baseline': 0.6103896,
 'auc': 0.7663121,
 'auc_precision_recall': 0.7127547,
 'average_loss': 0.56688124,
 'label/mean': 0.38961038,
 'loss': 130.94957,
 'precision': 0.71875,
 'prediction/mean': 0.35251445,
 'recall': 0.51111114,
 'global_step': 1000}

In [22]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=1000,num_epochs=1,shuffle=False)
list(model.predict(eval_input_func))

In [23]:
list(result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\matth\AppData\Local\Temp\tmpwnbsqwtf\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'logits': array([-0.98663175], dtype=float32),
  'logistic': array([0.2715779], dtype=float32),
  'probabilities': array([0.7284221 , 0.27157786], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-0.47860706], dtype=float32),
  'logistic': array([0.3825811], dtype=float32),
  'probabilities': array([0.61741894, 0.3825811 ], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([0.11550164], dtype=float32),
  'logistic': array([0.52884334], dtype=float32),
  'probabilities': array([0.47115666, 0.52884334], dtype=float32),
  'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([-0.7230127], dtype=float32),
  'logistic': array([0.3267299], dtype=float32),
  'probabilities': array([0.6732701 , 0.32672992], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits

In [25]:
# embedded_group_cal = tf.feature_column.embedding_column(assigned_group,dimension=4)
# feat_cols = [num_preg,plasma_gluc,dias_press,tricep,insulin,bmi,diabetes_pedigree,embedded_group_cal,age_bucket]

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\matth\\AppData\\Local\\Temp\\tmp2yug0jha', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018E0C2391D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train,batch_size=10,num_epochs=1000,shuffle=True)
MLP_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10], feature_columns=feat_cols, n_classes=2)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\matth\\AppData\\Local\\Temp\\tmpaavolw8t', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018E0C23E2B0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
MLP_model.train(input_fn=input_func, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\matth\AppData\Local\Temp\tmpaavolw8t\model.ckpt.
INFO:tensorflow:loss = 7.2644815, step = 0
INFO:tensorflow:global_step/sec: 159.622
INFO:tensorflow:loss = 6.172939, step = 100 (0.627 sec)
INFO:tensorflow:global_step/sec: 216.249
INFO:tensorflow:loss = 5.881811, step = 200 (0.463 sec)
INFO:tensorflow:global_step/sec: 210.881
INFO:tensorflow:loss = 4.569417, step = 300 (0.475 sec)
INFO:tensorflow:global_step/sec: 193.61
INFO:tensorflow:loss = 4.9933114, step = 400 (0.517 sec)
INFO:tensorflow:global_step/sec: 191.133
INFO:tensorflow:loss = 7.6299047, step = 500 (0.522 sec)
INFO:tensorflow:global_step/sec: 188.908
INFO:tensorflow:loss = 6.254265, step = 600 (0.528 sec)
INFO:tensorflow:global_step/sec: 

In [29]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=1000,num_epochs=1,shuffle=False)
MLP_model.evaluate(input_fn=input_func, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-23-19:51:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\matth\AppData\Local\Temp\tmpaavolw8t\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2019-02-23-19:51:23
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7903, accuracy_baseline = 0.6681, auc = 0.86370265, auc_precision_recall = 0.7644372, average_loss = 0.44225448, global_step = 1000, label/me

{'accuracy': 0.7903,
 'accuracy_baseline': 0.6681,
 'auc': 0.86370265,
 'auc_precision_recall': 0.7644372,
 'average_loss': 0.44225448,
 'label/mean': 0.3319,
 'loss': 4.422545,
 'precision': 0.66639435,
 'prediction/mean': 0.39634255,
 'recall': 0.73727024,
 'global_step': 1000}